# GENERA IMMAGINI

In [ ]:
# Install Kaggle API if not already installed
%pip install -q kaggle

# Upload your kaggle.json file
from google.colab import files
files.upload()  # Upload kaggle.json here

# Move kaggle.json to the correct location and set permissions
%mkdir -p ~/.kaggle
%mv kaggle.json ~/.kaggle/
%chmod 600 ~/.kaggle/kaggle.json

# Download the dataset from Kaggle
# Replace 'username/dataset-name' with the actual Kaggle dataset name
# !kaggle datasets download -d denislukovnikov/ffhq256-images-only -p /content

# Unzip the dataset
import zipfile
import os

dataset_zip_path = '/content/ffhq256-images-only.zip'  # Change this to the actual downloaded file name
dataset_extracted_dir = '/content/dataset_images'

if not os.path.exists(dataset_extracted_dir):
    print("Extracting dataset...")
    with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
        zip_ref.extractall(dataset_extracted_dir)
    print("Dataset extracted.")
else:
    print("Dataset already extracted.")

# Now the dataset is available; continue with image processing code

import PIL
import torch
import matplotlib.pyplot as plt
from diffusers import StableDiffusionInstructPix2PixPipeline, EulerAncestralDiscreteScheduler

# Initialize the model pipeline
model_id = "timbrooks/instruct-pix2pix"
pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(model_id, torch_dtype=torch.float16, safety_checker=None)
pipe.to("cuda")
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

# Define output zip paths for storing original and modified images
original_zip_path = '/content/original_images.zip'
modified_zip_path = '/content/modified_images.zip'

# Function to load a local image
def load_image_from_local(path):
    image = PIL.Image.open(path)
    image = PIL.ImageOps.exif_transpose(image)  # Handle orientation based on EXIF data
    image = image.convert("RGB")                # Convert to RGB format if needed
    return image

# Define prompt for style transfer
prompt = "vincent van gogh style"

# Process and save images
processed_images = []
max_images = 1000  # Adjust max_images if needed
count = 0

# Open zip files to save images
with zipfile.ZipFile(original_zip_path, 'w') as original_zip, zipfile.ZipFile(modified_zip_path, 'w') as modified_zip:
    for dirname, _, filenames in os.walk(dataset_extracted_dir):
        for filename in filenames:
            if count >= max_images:
                break

            image_path = os.path.join(dirname, filename)
            print(f"Number {count}, Processing image: {image_path}")

            # Load the original image
            original_image = load_image_from_local(image_path)

            # Save the original image to the zip file
            with original_zip.open(filename, 'w') as img_file:
                original_image.save(img_file, format='PNG')

            # Apply style transfer to the image
            modified_images = pipe(prompt, image=original_image).images
            modified_image = modified_images[0]

            # Save the modified image to the modified zip file
            with modified_zip.open(filename, 'w') as img_file:
                modified_image.save(img_file, format='PNG')

            # Store the modified image for display
            processed_images.append(modified_image)
            
            count += 1

        if count >= max_images:
            break

print("Processing complete. Zipped files saved at:")
print(f"- Original images: {original_zip_path}")
print(f"- Modified images: {modified_zip_path}")


# TEST PIX2PIX TRANSFER LEARNING

In [ ]:
!pip install git+https://github.com/tensorflow/examples.git
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import tensorflow as tf
from tensorflow_examples.models.pix2pix import pix2pix
import numpy as np
from PIL import Image
import os

class AdaptationLayer(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.adaptation = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.InstanceNorm2d(channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        return self.adaptation(x)

class PyTorchGenerator(nn.Module):
    def __init__(self, tf_generator):
        super().__init__()
        self.tf_generator = tf_generator
        
        # Add trainable adaptation layers
        self.input_adaptation = AdaptationLayer(3)
        self.output_adaptation = AdaptationLayer(3)
        
    def forward(self, x):
        # Apply input adaptation
        x = self.input_adaptation(x)
        
        # Convert PyTorch tensor to TF tensor, with detach
        x_tf = tf.convert_to_tensor(x.detach().cpu().numpy().transpose(0, 2, 3, 1))
        
        # Run through frozen TF model
        output_tf = self.tf_generator(x_tf, training=False)
        
        # Convert back to PyTorch tensor
        output_torch = torch.from_numpy(output_tf.numpy().transpose(0, 3, 1, 2))
        if x.is_cuda:
            output_torch = output_torch.cuda()
        
        # Apply output adaptation
        output_torch = self.output_adaptation(output_torch)
        
        return output_torch

class PyTorchDiscriminator(nn.Module):
    def __init__(self, tf_discriminator):
        super().__init__()
        self.tf_discriminator = tf_discriminator
        
        # Add trainable adaptation layers
        self.input_adaptation = AdaptationLayer(6)  # 6 channels for concatenated input
        self.output_adaptation = nn.Conv2d(1, 1, 1)  # 1x1 conv for final output
        
    def forward(self, x, y):
        # Concatenate input and target/generated images
        xy = torch.cat([x, y], dim=1)
        
        # Apply input adaptation
        xy = self.input_adaptation(xy)
        
        # Convert PyTorch tensor to TF tensor, with detach
        xy_tf = tf.convert_to_tensor(xy.detach().cpu().numpy().transpose(0, 2, 3, 1))
        
        # Split back into x and y for TF discriminator
        x_tf = xy_tf[..., :3]
        y_tf = xy_tf[..., 3:]
        
        # Run through frozen TF model
        output_tf = self.tf_discriminator([x_tf, y_tf], training=False)
        
        # Convert back to PyTorch tensor
        output_torch = torch.from_numpy(output_tf.numpy()).permute(0, 3, 1, 2)
        if x.is_cuda:
            output_torch = output_torch.cuda()
        
        # Apply output adaptation
        output_torch = self.output_adaptation(output_torch)
        
        return output_torch

class Pix2PixDataset(Dataset):
    def __init__(self, input_folder, target_folder, image_size=(256, 256)):
        self.input_paths = sorted([os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith('.png')])
        self.target_paths = sorted([os.path.join(target_folder, f) for f in os.listdir(target_folder) if f.endswith('.png')])
        self.transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    def __len__(self):
        return len(self.input_paths)

    def __getitem__(self, idx):
        input_image = Image.open(self.input_paths[idx]).convert('RGB')
        target_image = Image.open(self.target_paths[idx]).convert('RGB')
        
        input_tensor = self.transform(input_image)
        target_tensor = self.transform(target_image)
        
        return input_tensor, target_tensor

def train_pix2pix(generator, discriminator, dataloader, num_epochs=100, device='cuda'):
    criterion_gan = nn.BCEWithLogitsLoss()
    criterion_l1 = nn.L1Loss()
    
    # Only optimize the adaptation layers
    optimizer_g = optim.Adam([
        {'params': generator.input_adaptation.parameters()},
        {'params': generator.output_adaptation.parameters()}
    ], lr=2e-4, betas=(0.5, 0.999))
    
    optimizer_d = optim.Adam([
        {'params': discriminator.input_adaptation.parameters()},
        {'params': discriminator.output_adaptation.parameters()}
    ], lr=2e-4, betas=(0.5, 0.999))
    
    generator.train()
    discriminator.train()
    
    for epoch in range(num_epochs):
        for i, (real_a, real_b) in enumerate(dataloader):
            real_a = real_a.to(device)
            real_b = real_b.to(device)
            batch_size = real_a.size(0)
            
            # Ground truths
            valid = torch.ones((batch_size, 1, 30, 30), requires_grad=False).to(device)
            fake = torch.zeros((batch_size, 1, 30, 30), requires_grad=False).to(device)
            
            # Generator
            optimizer_g.zero_grad()
            fake_b = generator(real_a)
            pred_fake = discriminator(real_a, fake_b)
            loss_gan = criterion_gan(pred_fake, valid)
            loss_l1 = criterion_l1(fake_b, real_b) * 30
            loss_g = loss_gan + loss_l1
            loss_g.backward()
            optimizer_g.step()
            
            # Discriminator
            optimizer_d.zero_grad()
            pred_real = discriminator(real_a, real_b)
            pred_fake = discriminator(real_a, fake_b.detach())
            loss_real = criterion_gan(pred_real, valid)
            loss_fake = criterion_gan(pred_fake, fake)
            loss_d = (loss_real + loss_fake) / 2
            loss_d.backward()
            optimizer_d.step()
            
        print(f"[Epoch {epoch}/{num_epochs}][D loss: {loss_d.item():.4f}] [G loss: {loss_g.item():.4f}]")

# Load the pre-trained TensorFlow Pix2Pix model
def load_tf_model():
    generator = pix2pix.unet_generator(output_channels=3, norm_type='instancenorm')
    discriminator = pix2pix.discriminator(norm_type='instancenorm', target=True)
    return generator, discriminator


if __name__ == '__main__':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load TF models
    tf_generator, tf_discriminator = load_tf_model()
    
    # Create PyTorch wrapper models with adaptation layers
    generator = PyTorchGenerator(tf_generator).to(device)
    discriminator = PyTorchDiscriminator(tf_discriminator).to(device)
    
    # Freeze TF model weights (they're already frozen in the TF model)
    
    # Load dataset
    dataset = Pix2PixDataset(
        input_folder='/kaggle/input/originalimages/original_images',
        target_folder='/kaggle/input/modifiedimages/modified_images'
    )
    dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=4)
    
    # Train adaptation layers
    train_pix2pix(generator, discriminator, dataloader, num_epochs=100, device=device)
    
    # Save fine-tuned models
    torch.save({
        'generator_state_dict': generator.state_dict(),
        'discriminator_state_dict': discriminator.state_dict(),
    }, 'fine_tuned_models.pth')